In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
fiona.supported_drivers


{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [2]:
# Load ACS data and Eviction data
df_acs = pd.read_csv('data/acs_data_5y_2021.csv')
df_acs_2016 = pd.read_csv('data/acs_data_5y_2016.csv')
df_evic = gpd.read_file('data/eviction_count_bg_2021.geojson')

In [3]:
print(df_acs.shape)
print(df_acs_2016.shape)
print(df_evic.shape)

(1570, 79)
(1669, 6)
(151583, 42)


In [4]:
# Select required columns and convert df_evic to eviction count dataframe by CBG
df_evic2 = df_evic[['GEOID','geometry','case_number','amount_filed']]
df_evic3 = df_evic2.copy()

# Remove rows with 'amount_filed' = 'Not Non-Payment of Rent'
df_evic2 = df_evic2[df_evic2['amount_filed'] != 'Not Non-Payment of Rent']

# Select rows with 'amount_filed' = 'Not Non-Payment of Rent'
df_evic3 = df_evic3[df_evic3['amount_filed'] == 'Not Non-Payment of Rent']

# Convert 'amount_filed' to numeric
df_evic2['amount_filed'] = pd.to_numeric(df_evic2['amount_filed'])

# Aggregate eviction count and average amount_filed by CBG
df_evic2 = df_evic2.dissolve(by='GEOID', aggfunc={'case_number':'count', 'amount_filed':'mean'})
df_evic3 = df_evic3.drop(columns=['amount_filed'])
df_evic3 = df_evic3.dissolve(by='GEOID', aggfunc={'case_number':'count'})

In [5]:
# Add leading zeros to tract column
df_acs['tract'] = df_acs['tract'].astype(str)
df_acs_2016['tract'] = df_acs_2016['tract'].astype(str)
df_acs['tract'] = df_acs['tract'].str.zfill(6)
df_acs_2016['tract'] = df_acs_2016['tract'].str.zfill(6)

# Convert state, county, tract, block group to string
df_acs['state'] = df_acs['state'].astype(str)
df_acs['county'] = df_acs['county'].astype(str)
df_acs['block group'] = df_acs['block group'].astype(str)

df_acs_2016['state'] = df_acs_2016['state'].astype(str)
df_acs_2016['county'] = df_acs_2016['county'].astype(str)
df_acs_2016['block group'] = df_acs_2016['block group'].astype(str)

# Create GEOID column
df_acs['GEOID'] = df_acs['state'] + df_acs['county'] + df_acs['tract'] + df_acs['block group']
df_acs_2016['GEOID'] = df_acs_2016['state'] + df_acs_2016['county'] + df_acs_2016['tract'] + df_acs_2016['block group']

# Create GEOID_tract column
df_acs['GEOID_tract'] = df_acs['state'] + df_acs['county'] + df_acs['tract']
df_acs_2016['GEOID_tract'] = df_acs_2016['state'] + df_acs_2016['county'] + df_acs_2016['tract']

# Change the column names in df_acs_2016 to match df_acs
df_acs_2016 = df_acs_2016.rename(columns={'median_gross_rent':'median_gross_rent_2016'})

In [6]:
# Compare median rent in 2016 and 2021 at the tract-level 
# (At the census block group level, it is challenging to clearly identify the change in boundaries)
# Create a tract-level median gross rent dataframe for 2016 and 2021
df_acs_tract_2016 = df_acs_2016[['GEOID_tract','median_gross_rent_2016']].copy()
df_acs_tract_2021 = df_acs[['GEOID_tract','median_gross_rent']].copy()

# Replace negative values with NaN
df_acs_tract_2016['median_gross_rent_2016'][df_acs_tract_2016['median_gross_rent_2016'] < 0] = np.nan
df_acs_tract_2021['median_gross_rent'][df_acs_tract_2021['median_gross_rent'] < 0] = np.nan

# Group by tract and sum
df_acs_tract_2016 = df_acs_tract_2016.groupby('GEOID_tract').sum().reset_index()
df_acs_tract_2021 = df_acs_tract_2021.groupby('GEOID_tract').sum().reset_index()

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_16203/1978444421.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_acs_tract_2016['median_gross_rent_2016'][df_acs_tract_2016['median_gross_rent_2016'] < 0] = np.nan
/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_16203/1978444421.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_acs_tract_2021['median_gross_rent'][df_acs_tract_2021['median_gross_rent'] < 0] = np.nan


In [7]:
# Read census tract relationship file between 2010 and 2020
file_path = 'https://www2.census.gov/geo/docs/maps-data/data/rel2020/tract/tab20_tract20_tract10_natl.txt'
df_tract_rel = pd.read_csv(file_path, sep='|')

# Select only Dallas County, Texas
df_tract_rel['GEOID_TRACT_20'] = df_tract_rel['GEOID_TRACT_20'].astype(str)
df_tract_rel['GEOID_TRACT_10'] = df_tract_rel['GEOID_TRACT_10'].astype(str)
df_tract_rel = df_tract_rel[df_tract_rel['GEOID_TRACT_20'].str.startswith('48113')]

# Merge df_acs_tract_2021 and df_acs_tract_2016 with df_tract_rel
df_tract_temp = pd.merge(df_acs_tract_2021, df_tract_rel[['GEOID_TRACT_20', 'GEOID_TRACT_10']], left_on='GEOID_tract', right_on='GEOID_TRACT_20')
df_tract_temp2 = pd.merge(df_tract_temp, df_acs_tract_2016, left_on='GEOID_TRACT_10', right_on='GEOID_tract')
df_tract_temp2 = df_tract_temp2[['GEOID_tract_x','median_gross_rent','median_gross_rent_2016']].groupby('GEOID_tract_x').mean()

# Change the column name
df_tract_temp2 = df_tract_temp2.rename(columns={'median_gross_rent':'median_gross_rent_tract_2021', 'median_gross_rent_2016':'median_gross_rent_tract_2016'})

# Merge df_acs and df_tract_temp2
df_acs = pd.merge(df_acs, df_tract_temp2, left_on='GEOID_tract', right_on='GEOID_tract_x', how='left')

In [8]:
# Merge ACS and Eviction dataframes
df_evic2.reset_index(inplace=True)
df_evic2['GEOID'] = df_evic2['GEOID'].astype(str)

df_evic3.reset_index(inplace=True)
df_evic3['GEOID'] = df_evic3['GEOID'].astype(str)

# Merge df_acs and df_evic['geometry] first before merging with df_evic2
df_acs2 = pd.merge(df_acs, df_evic[['GEOID','geometry']].drop_duplicates(), on='GEOID', how='left')

# Merge df_acs and df_evic2
df = pd.merge(df_acs2, df_evic2, on='GEOID', how='left')

# Merge df_acs and df_evic3 (non-payment(np) of rent)
df_np = pd.merge(df_acs2, df_evic3, on='GEOID', how='left')

In [9]:
# Select required columns
required_cols = ['GEOID','geometry_x','case_number', 'amount_filed', 'total_population', 'median_age' ,
                 'race_white', 'race_black', 'race_asian', 'race_hispanic','hh_total', 'hh_w_child_total', 
                 'edu_att_over25_no_schooling_completed', 'edu_att_over25_highschool_graduate', 
                 'edu_att_over25_master_degree', 'edu_att_over25_professional_degree',
                 'edu_att_over25_doctorate_degree', 'hher_female',
                 'no_internet_access', 'housing_unit',
                 'housing_vacant', 'tenure_renter_occupied', 'vacancy_for_rent', 'median_year_built', 
                 'housing_median_value', 'ind_poverty', 'hh_median_income', 'median_gross_rent', 
                 'median_gross_rent_tract_2021', 'median_gross_rent_tract_2016',
                 'mortgage_status_total',	'mortgage_status_w_mortgage','median_monthly_owner_costs',	
                 'median_monthly_owner_costs_w_mortgage', 'hh_nonfamily', 'hh_average_size_renter_occupied',
                 '1unit_detached_structure', '1unit_attached_structure','gross_rent_percent_hhincome_total',
                 'gross_rent_percent_hhincome_40to50', 'gross_rent_percent_hhincome_50ormore',
                 'ind_labor_force', 'ind_labor_force_civilian_unemployed',
                 'hh_w_ssi', 'hh_w_pai', 'hh_w_foodstamp_SNAP',
                 'time_to_work_total', 'time_to_work_lt5', 'time_to_work_5to9',
                 'time_to_work_10to14', 'time_to_work_15to19', 'time_to_work_20to24', 
                 'time_to_work_25to29', 'time_to_work_30to34', 'time_to_work_35to39', 
                 'time_to_work_40to44', 'time_to_work_45to59', 'time_to_work_60to89', 'time_to_work_mt90'
                 ]
df = df[required_cols]

In [10]:
# Select required columns (without amount_filed)
required_cols = ['GEOID','geometry_x','case_number', 'total_population', 'median_age' ,
                 'race_white', 'race_black', 'race_asian', 'race_hispanic','hh_total', 'hh_w_child_total', 
                 'edu_att_over25_no_schooling_completed', 'edu_att_over25_highschool_graduate', 
                 'edu_att_over25_master_degree', 'edu_att_over25_professional_degree',
                 'edu_att_over25_doctorate_degree', 'hher_female',
                 'no_internet_access', 'housing_unit', 
                 'housing_vacant', 'tenure_renter_occupied', 'vacancy_for_rent', 'median_year_built', 
                 'housing_median_value', 'ind_poverty', 'hh_median_income', 'median_gross_rent',
                 'median_gross_rent_tract_2021', 'median_gross_rent_tract_2016',
                 'mortgage_status_total',	'mortgage_status_w_mortgage','median_monthly_owner_costs',	
                 'median_monthly_owner_costs_w_mortgage', 'hh_nonfamily', 'hh_average_size_renter_occupied',
                 '1unit_detached_structure', '1unit_attached_structure','gross_rent_percent_hhincome_total',
                 'gross_rent_percent_hhincome_40to50', 'gross_rent_percent_hhincome_50ormore',
                 'ind_labor_force', 'ind_labor_force_civilian_unemployed',
                 'hh_w_ssi', 'hh_w_pai', 'hh_w_foodstamp_SNAP',
                 'time_to_work_total', 'time_to_work_lt5', 'time_to_work_5to9',
                 'time_to_work_10to14', 'time_to_work_15to19', 'time_to_work_20to24', 
                 'time_to_work_25to29', 'time_to_work_30to34', 'time_to_work_35to39', 
                 'time_to_work_40to44', 'time_to_work_45to59', 'time_to_work_60to89', 'time_to_work_mt90'
                 ]
df_np = df_np[required_cols]

In [11]:
# Separate df into integer, float and string dataframes
df_int = df.select_dtypes(include=['int64'])
df_str = df.select_dtypes(include=['object'])
df_float = df.select_dtypes(include=['float64'])

df_np_int = df_np.select_dtypes(include=['int64'])
df_np_str = df_np.select_dtypes(include=['object'])
df_np_float = df_np.select_dtypes(include=['float64'])

# Replace negative values with NaN
df_int[df_int < 0] = np.nan
df_float[df_float < 0] = np.nan

df_np_int[df_np_int < 0] = np.nan
df_np_float[df_np_float < 0] = np.nan

In [12]:
# Concatenate integer, float and string dataframes
df2 = pd.concat([df['geometry_x'], df_int, df_float, df_str], axis=1)
df_np2 = pd.concat([df_np['geometry_x'], df_np_int, df_np_float, df_np_str], axis=1)

In [13]:
# Create new columns: eviction rate
df2['eviction_rate'] = (df2['case_number'] / 5) / df2['tenure_renter_occupied']

# Create new columns: education level less than high school
df2['edu_lt_highschool'] = (df2['edu_att_over25_no_schooling_completed'] + df2['edu_att_over25_highschool_graduate'])/df2['total_population']

# Create new columns: education level with graduate degree
df2['edu_grad'] = (df2['edu_att_over25_master_degree'] + df2['edu_att_over25_professional_degree'] + df2['edu_att_over25_doctorate_degree'])/df2['total_population']

# Create new columns: black people percentage
df2['black_ratio'] = df2['race_black'] / df2['total_population']

# Create new columns: white people percentage
df2['white_ratio'] = df2['race_white'] / df2['total_population']

# Create new columns: asian people percentage
df2['asian_ratio'] = df2['race_asian'] / df2['total_population']

# Create new columns: hispanic people percentage
df2['hispanic_ratio'] = df2['race_hispanic'] / df2['total_population']

# Create new columns: household with children percentage
df2['hh_w_child_ratio'] = df2['hh_w_child_total'] / df2['hh_total']

# Create new columns: household with female householder
df2['hher_female_ratio'] = df2['hher_female'] / df2['hh_total']

# Create new columns: renter occupied housing unit percentage
df2['renter_occ_rate'] = df2['tenure_renter_occupied'] / df2['housing_unit']

# Create new columns: vacant housing unit percentage
df2['vacancy_rate'] = df2['housing_vacant'] / df2['housing_unit']

# Create new columns: unemployment rate
df2['unemployment_rate'] = df2['ind_labor_force_civilian_unemployed'] / df2['ind_labor_force']

# Create new columns: poverty rate
df2['poverty_rate'] = df2['ind_poverty'] / df2['total_population']

# Create new columns: single unit structure percentage
df2['1unit_structure_ratio'] = (df2['1unit_detached_structure'] + df2['1unit_attached_structure']) / df2['housing_unit']

# Create new columns: household ratio with social programs
df2['hh_social_programs'] = (df2['hh_w_ssi'] + df2['hh_w_pai'] + df2['hh_w_foodstamp_SNAP'])/df2['hh_total']

# Create new columns: morgage status percentage
df2['mortgage_status_ratio'] = df2['mortgage_status_w_mortgage'] / df2['mortgage_status_total']

# Create new columns: gross rent percentage
df2['gross_rent_mt40'] = (df2['gross_rent_percent_hhincome_40to50'] + df2['gross_rent_percent_hhincome_50ormore'])/df2['gross_rent_percent_hhincome_total']

# Create new columns: household with nonfamily ratio
df2['hh_nonfamily_ratio'] = df2['hh_nonfamily'] / df2['hh_total']

# Create new columns: time to work less than 30 mins
df2['time_to_work_lt30'] = (df2['time_to_work_lt5'] + df2['time_to_work_5to9'] + df2['time_to_work_10to14'] + df2['time_to_work_15to19'] + df2['time_to_work_20to24'] + df2['time_to_work_25to29'])/df2['time_to_work_total']

# Create new columns: time to work between 30 and 59 mins
df2['time_to_work_30to59'] = (df2['time_to_work_30to34'] + df2['time_to_work_35to39'] + df2['time_to_work_40to44'] + df2['time_to_work_45to59'])/df2['time_to_work_total']

# Create new columns: time to work more than 60 mins
df2['time_to_work_mt60'] = (df2['time_to_work_60to89'] + df2['time_to_work_mt90'])/df2['time_to_work_total']

# Create new columns: median gross rent change between 2016 and 2021
df2['median_gross_rent_tract_2016'] = df2['median_gross_rent_tract_2016'] + 1
df2['median_gross_rent_tract_2021'] = df2['median_gross_rent_tract_2021'] + 1
df2['median_gross_rent_change'] = (df2['median_gross_rent_tract_2021'] - df2['median_gross_rent_tract_2016'])/df2['median_gross_rent_tract_2016']

In [14]:
# Create new columns: eviction rate
df_np2['eviction_rate'] = (df_np2['case_number'] / 5) / df_np2['tenure_renter_occupied']

# Create new columns: education level less than high school
df_np2['edu_lt_highschool'] = (df_np2['edu_att_over25_no_schooling_completed'] + df_np2['edu_att_over25_highschool_graduate'])/df_np2['total_population']

# Create new columns: education level with graduate degree
df_np2['edu_grad'] = (df_np2['edu_att_over25_master_degree'] + df_np2['edu_att_over25_professional_degree'] + df_np2['edu_att_over25_doctorate_degree'])/df_np2['total_population']

# Create new columns: black people percentage
df_np2['black_ratio'] = df_np2['race_black'] / df_np2['total_population']

# Create new columns: white people percentage
df_np2['white_ratio'] = df_np2['race_white'] / df_np2['total_population']

# Create new columns: asian people percentage
df_np2['asian_ratio'] = df_np2['race_asian'] / df_np2['total_population']

# Create new columns: hispanic people percentage
df_np2['hispanic_ratio'] = df_np2['race_hispanic'] / df_np2['total_population']

# Create new columns: household with children percentage
df_np2['hh_w_child_ratio'] = df_np2['hh_w_child_total'] / df_np2['hh_total']

# Create new columns: household with female householder
df_np2['hher_female_ratio'] = df_np2['hher_female'] / df_np2['hh_total']

# Create new columns: renter occupied housing unit percentage
df_np2['renter_occ_rate'] = df_np2['tenure_renter_occupied'] / df_np2['housing_unit']

# Create new columns: vacant housing unit percentage
df_np2['vacancy_rate'] = df_np2['housing_vacant'] / df_np2['housing_unit']

# Create new columns: unemployment rate
df_np2['unemployment_rate'] = df_np2['ind_labor_force_civilian_unemployed'] / df_np2['ind_labor_force']

# Create new columns: poverty rate
df_np2['poverty_rate'] = df_np2['ind_poverty'] / df_np2['total_population']

# Create new columns: single unit structure percentage
df_np2['1unit_structure_ratio'] = (df_np2['1unit_detached_structure'] + df_np2['1unit_attached_structure']) / df_np2['housing_unit']

# Create new columns: household ratio with social programs
df_np2['hh_social_programs'] = (df_np2['hh_w_ssi'] + df_np2['hh_w_pai'] + df_np2['hh_w_foodstamp_SNAP'])/df_np2['hh_total']

# Create new columns: morgage status percentage
df_np2['mortgage_status_ratio'] = df_np2['mortgage_status_w_mortgage'] / df_np2['mortgage_status_total']

# Create new columns: gross rent percentage
df_np2['gross_rent_mt40'] = (df_np2['gross_rent_percent_hhincome_40to50'] + df_np2['gross_rent_percent_hhincome_50ormore'])/df_np2['gross_rent_percent_hhincome_total']

# Create new columns: household with nonfamily ratio
df_np2['hh_nonfamily_ratio'] = df_np2['hh_nonfamily'] / df_np2['hh_total']

# Create new columns: time to work less than 30 mins
df_np2['time_to_work_lt30'] = (df_np2['time_to_work_lt5'] + df_np2['time_to_work_5to9'] + df_np2['time_to_work_10to14'] + df_np2['time_to_work_15to19'] + df_np2['time_to_work_20to24'] + df_np2['time_to_work_25to29'])/df_np2['time_to_work_total']

# Create new columns: time to work between 30 and 59 mins
df_np2['time_to_work_30to59'] = (df_np2['time_to_work_30to34'] + df_np2['time_to_work_35to39'] + df_np2['time_to_work_40to44'] + df_np2['time_to_work_45to59'])/df_np2['time_to_work_total']

# Create new columns: time to work more than 60 mins
df_np2['time_to_work_mt60'] = (df_np2['time_to_work_60to89'] + df_np2['time_to_work_mt90'])/df_np2['time_to_work_total']

# Create new columns: median gross rent change between 2016 and 2021
df_np2['median_gross_rent_tract_2016'] = df_np2['median_gross_rent_tract_2016'] + 1
df_np2['median_gross_rent_tract_2021'] = df_np2['median_gross_rent_tract_2021'] + 1
df_np2['median_gross_rent_change'] = (df_np2['median_gross_rent_tract_2021'] - df_np2['median_gross_rent_tract_2016'])/df_np2['median_gross_rent_tract_2016']

In [15]:
df_np2['median_gross_rent_change'].describe()

count    1570.000000
mean       31.612713
std       351.567155
min        -0.999823
25%        -0.335845
50%         0.015125
75%         0.377316
max      5168.000000
Name: median_gross_rent_change, dtype: float64

In [18]:
print(df2.shape)
print(df_np2.shape)

(1547, 81)
(1547, 80)


In [17]:
# Remove rows with zero total population
df2 = df2[df2['total_population']!=0]
df_np2 = df_np2[df_np2['total_population']!=0]

# Remove rows with zero housing unit
df2 = df2[df2['housing_unit']!=0]
df_np2 = df_np2[df_np2['housing_unit']!=0]

# Replace NaN with zero for 'gross_rent_mt40' if 'tenure_renter_occupied' = 0
df2.loc[df2['tenure_renter_occupied']==0, 'gross_rent_mt40'] = 0
df_np2.loc[df_np2['tenure_renter_occupied']==0, 'gross_rent_mt40'] = 0

# Replace NaN with zero for 'mortgate_status_ratio' if 'mortgage_status_total' = 0
df2.loc[df2['mortgage_status_total']==0, 'mortgage_status_ratio'] = 0
df_np2.loc[df_np2['mortgage_status_total']==0, 'mortgage_status_ratio'] = 0

# Fill NaN with 0 for 'case_number' and 'amount_filed'
df2['case_number'].fillna(0, inplace=True)
df_np2['case_number'].fillna(0, inplace=True)
df2['amount_filed'].fillna(0, inplace=True)

In [19]:
df2.to_csv('data/acs_evic_data_wo_zero_pop.csv')
df_np2.to_csv('data/acs_evic_data_np_wo_zero_pop.csv')